In [1]:
!pip install geopy

     |████████████████████████████████| 112kB 7.2MB/s eta 0:00:01


In [2]:
import numpy as np 

import pandas as pd 

from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


In [3]:
# getting Toronto Lattitude and Longitude 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


1. Import CSV file created in the previous step of the project

In [4]:
dfToronto=pd.read_csv('dfToronto.csv', index_col=[0])

In [5]:
dfToronto.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.786947,-79.385975


2. Plot the Toronto map with the neighborhoods marked on it with blue dots

In [6]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#
for lat, lng, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

3. Defining Foursquare credentials

In [7]:
CLIENT_ID = '*****************' # your Foursquare ID
CLIENT_SECRET = '*******************' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: **************
CLIENT_SECRET:************


4. Getting the venues in 500 meter radius of each neighborhood

4.1 getting Long&Lat for one neighborhood

In [8]:
neighborhood_latitude = dfToronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.7869473, -79.385975.


In [9]:
radius=500
LIMIT=100

In [10]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [11]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results=requests.get(url).json()

In [12]:
len(results['response']['groups'][0]['items'])

4

4.2 Defining a function to get the venues for every neighborhood 

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
Toronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

In [15]:
print(Toronto_venues.shape)
Toronto_venues.head()

(758, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
1,The Beaches,43.786947,-79.385975,TD Canada Trust,43.788074,-79.380367,Bank
2,The Beaches,43.786947,-79.385975,Maxim's Cafe and Patisserie,43.787863,-79.380751,Café
3,The Beaches,43.786947,-79.385975,Kaga Sushi,43.787758,-79.381090,Japanese Restaurant
4,"The Danforth West , Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [16]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,2,2,2,2,2,2
"Brockton , Parkdale Village , Exhibition Place",40,40,40,40,40,40
Business reply mail Processing CentrE,4,4,4,4,4,4
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",17,17,17,17,17,17
Central Bay Street,7,7,7,7,7,7
Christie,3,3,3,3,3,3
Church and Wellesley,7,7,7,7,7,7
"Commerce Court , Victoria Hotel",1,1,1,1,1,1
Davisville,3,3,3,3,3,3


In [17]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 200 uniques categories.


5. Preparing data bdfore modeling

In [18]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
#Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
Toronto_onehot.drop('Neighborhood',axis=1, inplace=True)

In [23]:
#Insert the Neighborhood column to the first column
Toronto_onehot.insert(loc=0, column='Neighborhood', value=Toronto_venues['Neighborhood'] )

In [24]:
Toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
Toronto_onehot.shape

(758, 200)

In [26]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.025,0.0,0.0,0.025000
2,Business reply mail Processing CentrE,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.058824
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000


6. Exploring the neighborhoods venue data 

In [27]:
#Prin 5 most frequent venues in ech neighborhood
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                       venue  freq
0          Martial Arts Dojo   0.5
1                  Cafeteria   0.5
2                    Airport   0.0
3         Miscellaneous Shop   0.0
4  Latin American Restaurant   0.0


----Brockton , Parkdale Village , Exhibition Place----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----Business reply mail Processing CentrE----
               venue  freq
0        Pizza Place  0.25
1     Sandwich Place  0.25
2  Mobile Phone Shop  0.25
3           Bus Line  0.25
4            Airport  0.00


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2                 Spa  0.06
3          Comic Shop  0.06
4                Park  0.06


----Central Bay Street----
         

In [28]:
#Define a function to get the most commone venues for each neighborhood 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Martial Arts Dojo,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Latin American Restaurant,Middle Eastern Restaurant,Bookstore
2,Business reply mail Processing CentrE,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Yoga Studio,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop
3,"CN Tower , King and Spadina , Railway Lands , ...",Light Rail Station,Yoga Studio,Spa,Fast Food Restaurant,Farmers Market,Comic Shop,Garden Center,Park,Pizza Place,Burrito Place
4,Central Bay Street,Coffee Shop,Pharmacy,Bank,Home Service,Pizza Place,Grocery Store,Butcher,Construction & Landscaping,College Stadium,Doner Restaurant


7. Run K-means clustering on the data 

In [30]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [31]:
Toronto_grouped_clustering

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
1,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.050000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.025000,0.000000,0.00,0.025000
2,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
3,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.058824
4,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
5,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
6,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
7,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
8,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000
9,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,0.017544,...,0.000000,0.000000,0.035088,0.00,0.0,0.052632,0.017544,0.000000,0.00,0.000000


In [88]:
#neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

In [32]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = dfToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last column

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Cosmetics Shop,Brewery,Bubble Tea Shop,Pub
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant,Café,Italian Restaurant,Gym,Bar,Pharmacy
3,M4M,East Toronto,Studio District,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Diner,Bookstore
4,M4N,Central Toronto,Lawrence Park,43.648198,-79.379817,0,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Deli / Bodega,Italian Restaurant,Japanese Restaurant,Seafood Restaurant


In [33]:
# create map of neighborhoods based on their lable 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
1,"The Danforth West , Riverdale",0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Cosmetics Shop,Brewery,Bubble Tea Shop,Pub
2,"India Bazaar , The Beaches West",0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant,Café,Italian Restaurant,Gym,Bar,Pharmacy
3,Studio District,0,Clothing Store,Coffee Shop,Café,Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Diner,Bookstore
4,Lawrence Park,0,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Deli / Bodega,Italian Restaurant,Japanese Restaurant,Seafood Restaurant
5,Davisville North,0,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bakery,Grocery Store,Bar,Gaming Cafe,Vegetarian / Vegan Restaurant,Dessert Shop
6,North Toronto West,0,Park,Field,Trail,Hockey Arena,Yoga Studio,Dance Studio,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
7,Davisville,0,Park,Construction & Landscaping,Bakery,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
8,"Moore Park , Summerhill East",0,Gift Shop,Italian Restaurant,Cuban Restaurant,Breakfast Spot,Dog Run,Restaurant,Bar,Dessert Shop,Eastern European Restaurant,Bookstore
9,"Summerhill West , Rathnelly , South Hill , For...",0,Coffee Shop,Hotel,Gym,Middle Eastern Restaurant,Mediterranean Restaurant,Burrito Place,Sandwich Place,Intersection,American Restaurant,Fried Chicken Joint


In [35]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Commerce Court , Victoria Hotel",1,Playground,Yoga Studio,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"St. James Town , Cabbagetown",2,Pizza Place,Yoga Studio,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


In [37]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Berczy Park,3,Cafeteria,Martial Arts Dojo,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner


In [38]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,4,Baseball Field,Construction & Landscaping,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner
29,"First Canadian Place , Underground city",4,Food Service,Baseball Field,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop


***From the results of the model we can see most of the neighborhoods in Toronto area are in one group and by looking at that group and other smaller groups ,  we cant find a very distinct geographical separation  based on the venues in each neighborhood